In [1]:
#决策树节点类
class decisionNode:
    #Impurity是一个广义的概念，用于表示数据集的混合程度和不确定性程度
    def __init__(self, feature, threshold, left, right, label, gini, is_leaf):
        self.feature = feature           #划分特征的维度
        self.threshold = threshold       #划分阈值
        self.left = left                 #左子树
        self.right = right               #右子树
        self.label = label               #叶节点值（标签）
        self.is_leaf = is_leaf             #是否为叶节点
        self.gini = gini                 #储存该节点的Gini Impurity

In [59]:

def getClasses(label):
    # 如果标签为空，则返回0
    if not label:
        return 0
    unique_classes = set(label.flatten())  # 使用 set 来获取不同类别
    num = len(unique_classes)
    return num


#计算叶节点值
def get_label(dataset):
    labels = dataset[:,-1]
    #初始化一个字典
    label_counts = {}
    #键是数据集中出现的类别，而值是对应类别出现的次数。
    for label in labels:                    #遍历数据集中的每个样本的标签
        if label in label_counts:           #检查当前标签是否已存在于字典中
            label_counts[label] += 1        #如果已存在，对应值加一，表示该类别出现的次数加一
        else:
            label_counts[label] = 1         #如果不存在，则添加该标签，并初始化值为一，表示该类别第一次出现
    
    max_count = max(label_counts.values())  #得到数据集中出现次数最多的类别的次数
    
    #‘largest_labels存储所有出现次数最多的类别’
    #循环遍历 label_counts 字典中的每个键值pair，
    #如果出现次数等于 max_count，则将对应的类别标签 label 添加到列表中。
    largest_labels = [label for label, count in label_counts.items() 
                      if count == max_count]
    #largest_labels 是存储了出现次数最多的类别的列表。
    #因为有可能存在多个类别出现次数相同的情况，所以 largest_labels 中可能包含了多个类别。
    
    #从出现次数最多的类别中选择第一个类别作为叶节点值
    #目的是确保每次运行算法时得到相同的结果，这样可以保持算法的可重复性和稳定性。
    return largest_labels[0]


#定义计算gini值的函数
def getGini(dataset):

    #dataset: the combination of data and label
    #dataset的最后一列是标签
    datalength = dataset.shape[0]       # dataset数据点的长度     
    #使用np.bincount得到每个类别出现的次数
    #label = dataset[:,-1]
    counts = np.bincount(dataset[:,-1].astype(int))
    probs  = counts / datalength        # 计算每个类别出现的概率
    gini = 1 - np.sum(probs ** 2)
    return gini


#划分用函数，用于划分后寻找最小gini值的划分阈值
def splitData(dataset, feature, threshold, type):
    datalength = dataset.shape[0]       #记录数据总长度
    channel = feature                   #当前划分特征维度

    subdataset = []                     #创建子数据集的空例，为后续储存数据做准备
    if type == 0:
        for i in range(datalength):
            #使用float函数确保比较的值是浮点数，而不是字符串或其他数据类型
            if float(dataset[i,feature]) <= threshold:      #储存小于等于阈值的数据为子数据集
                subdataset.append(dataset[i,:])                #储存整行数据
    elif type == 1:
        for i in range(datalength):
            if float(dataset[i,feature]) >  threshold :      #储存大于阈值的数据为子数据集
                subdataset.append(dataset[i,:])                #储存整行数据

    return np.array(subdataset), len(subdataset)

#找到最佳特征列
def bestFeature(dataset):

    datalength = dataset.shape[0]       #记录数据总长度
    numFeatures = len(dataset[0]) - 1   #特征总数，减一是因为dataset最后一列是标签
    #当只有一个特征时
    #if dataset.size < 0:
    #    bestFeature = 0
    #    return bestFeature
        
    #预设定gini值
    minGini = 1
    bestThreshold = []                      #创建空例，储存阈值
    bestFeature = []                        #创建空例，储存特征维度
    #start_time = time.time()

    for i in range(numFeatures):        #从第i个特征开始遍历所有特征
        feaColumn = list(dataset[:,i])  #提取第i个维度的数据列，并转化为列表，为了方便排序
        sorted_data = sorted(feaColumn) #对这列特征排序
        feaGini = 1                     #预设定该特征下使用阈值划分计算得到的最小gini值为1
        feaThres = []                   #创建空例，储存该特征下每个gini值对应的划分阈值
        
        for j in range(datalength - 1): #循环的目的是为了寻找gini值最小的划分阈值
            threshold = (sorted_data[j]+sorted_data[j+1])/2
            dataLeft , lenLeft  = splitData(dataset, i, threshold, 0)      #当前特征下，划分阈值划分的左子树
            dataRight, lenRight = splitData(dataset, i, threshold, 1)      #右子树 
            # 检查子数据集是否为空，如果是，则跳过当前阈值
            if lenLeft == 0 or lenRight == 0:
                continue
            #计算此分法对应Gini值
            GiniValue = (lenLeft/datalength) * getGini(dataLeft) + (lenRight/datalength) * getGini(dataRight)  

            if GiniValue < feaGini:   #如果新计算的gini值小于与预设值，更新gini和阈值
                feaGini = GiniValue    #储存这一循环中，划分阈值下计算的gini值
                feaThre = threshold    #储存这一循环中，划分阈值
            
        if feaGini < minGini:
            minGini = feaGini          #储存这一特征下最小的gini值
            bestThreshold = feaThre    #储存这一特征下，最小gini值对应的划分阈值
            bestFeature = i            #储存这一特征维度
        
    #print('特征：',bestFeature)   
    #print('一个划分特征所用时间 %fs!' % (time.time() - start_time))
    return bestFeature, bestThreshold, minGini



        
#创建决策树
def build_tree(dataset, depth, max_depth,min_samples_split):
    
    length = dataset.shape[0]
    max_depth = max_depth
    min_samples_split = min_samples_split
    #if depth == self.max_depth or length < self.min_samples_split:
    #    return node
    if dataset.size<=0:
        return None
    
    
    #得到最好的划分特征维度和划分阈值，以及对应的最小gini值
    feature, threshold, minGini = bestFeature(dataset)
    
    #按照划分特征的维度进行排序
    sorted_data = sorted(dataset, key=lambda x: x[feature])  

    
    #根据划分特征维度和阈值划分左右子树
    left_data = np.array([row for row in sorted_data if row[feature] <= threshold])
    right_data = np.array([row for row in sorted_data if row[feature] > threshold])
    leftLen = left_data.shape[0]
    rightLen = right_data.shape[0]
    #print ('左子树数据形状 ' , (leftLen))
    #print ('右子树数据形状 ' , (rightLen))
    left = None
    right = None
    #判断是否达到最大深度，或节点中的样本数是否小于min_samples_split时
    if depth < max_depth and leftLen > 1:
        # or length >= self.min_samples_split:
        #递归构建左右子树
        left = build_tree(left_data, depth + 1,
                          max_depth=max_depth, min_samples_split=min_samples_split)
    if depth < max_depth and rightLen >1:    
        right = build_tree(right_data, depth + 1,
                          max_depth=max_depth,min_samples_split=min_samples_split)
    #创建内部节点
    node = decisionNode(feature=feature, threshold=threshold, gini = minGini,
                        left =left, right = right, label = None,is_leaf=None)
    
    #判断是否达到最大深度，或节点中的样本数小于min_samples_split时，返回叶子节点的值
    if depth == max_depth or length <= min_samples_split:

        label = get_label(dataset)  # 获取叶节点值
        node.label = label
        node.is_leaf = True
    
    return node


#分类树类
#class Classifier:

#    def __init__(self):
        #用数据集构建决策树

        #self.min_samples_split = min_samples_split

        
        

    #决策树搜索
def searchTree(tree, point):

    #检查决策树是否存在，否则不执行
    if tree is None:                              
        return None
    
    #用于记录搜索路径上的节点信息
    stack = []

    #在决策树中搜索，并确定point会落在哪个叶子节点上
    current_node = tree                                         #储存根节点；决策树结构为根节点下左右子节点树，下一个循环后目标点更新成左节点或右节点，一直循环，直到目标点为none

    while current_node:                                              #当前节点根据循环不停向下递归查找最邻近的点，直到提取数据点为None结束循环
        stack.append(current_node)                                   #当前节点信息存储于堆栈中
        threshold = current_node.threshold                           #从当前节点中提取阈值
        feature = current_node.feature                               #从当前节点中提取划分特征维度
        
        if current_node.is_leaf == True:                             #判断当前节点是否为叶子节点
            return current_node.label                                #如果是，返回当前叶节点值为预测标签
        
        if point.size > 0 and point[feature] <= threshold:                            #比较当前维度下未知点和阈值的大小，判断走左子树还是右子树
            current_node = current_node.left
        else:
            current_node = current_node.right

def selectSamples(data, label, numberSamplePerClass):

    unique_classes = np.unique(label)  # 获取唯一的类别标签
    #初始化抽样后的数据和标签列表
    subData = []
    subLabel = []
    num = numberSamplePerClass

    #对每个类别进行遍历
    for i in unique_classes:
        #找到标签中当前类别的索引
        index = np.where(label == i)[0]
        
        #处理样本不足的情况，可以选择跳过该类别或者使用全部样本
        if len(index) < num:
            Idx = len(index)  # 使用全部样本
            selectedIdx = np.random.choice(index, size=Idx, replace=True)
        else:    
            #从当前类别的索引中随机抽样指定数量的样本,replace参数为True意味是有放回抽样
            selectedIdx = np.random.choice(index, size=num, replace=True)
        #将抽样后的数据和标签添加到列表中
        subData.extend(data[selectedIdx,:])
        subLabel.extend(label[selectedIdx])
    
            #将抽样后的数据和标签转换为numpy数组并返回
    subdata = np.array(subData)
    sublabel = np.array(subLabel)
    sublabel = sublabel.reshape([-1,1])
    #print(subdata.shape)
    #print(sublabel.shape)

    #将抽样后的数据和标签转换为numpy数组并返回
    return subdata, sublabel
        
#随机森林类
class RandomForest:

    def __init__(self, num_estimators, max_depth, min_samples_split, numberSamplePerClass):
        self.num_estimators = num_estimators           #储存决策树数量
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.trees = []                                #存储随机森林中的决策树
        self.numberSamplePerClass = numberSamplePerClass
        

    #用数据构建决策树
    def fit(self, data, label): 
        count=0
        for _ in range(self.num_estimators):    #循环使用下划线因为不关心循环变量的值，只是想重复执行创建决策树的操作
            #从数据集中有放回地随机抽样作为决策树训练数据
            start_time = time.time()
            subdata, sublabel = selectSamples(data, label, self.numberSamplePerClass)
            #将数据集和标签结合在一起
            dataset = np.hstack((subdata, sublabel)) 
            count = count+1
            print(count)
            subtree = build_tree(dataset, depth=0,max_depth=self.max_depth, min_samples_split=self.min_samples_split)            #构建决策树
            self.trees.append(subtree)             #储存决策树
            print('一棵树生成所用时间 %fs!' % (time.time() - start_time))
            
            
    #预测
    def predict(self, testData):
        #（暂时不用）datalength = testData.shape[0]
        predictions = []  #用于存储每个树的预测结果
        for point in testData:
            pointpredictions = []  #储存每棵树对该点的预测
            for tree in self.trees:#按循环以此使用每棵树
                prediction = searchTree(tree,point)
                pointpredictions.append(prediction)

            #对于每个数据点，返回多个决策树的结果投票得出预测值
            # set(pointpredictions)作用是去重，这样可以得到一个不重复的元素集合
            # max()函数结合key=pointpredictions.count参数可以找到在预测点中出现次数最多的元素
            label = max(set(pointpredictions), key=pointpredictions.count)
            predictions.append(label)

        #将预测结果列表转换为NumPy数组后，reshape成二维数组，方便之后与正确标签进行对比
        predictions_array = np.array(predictions)
        predictions_reshaped = np.reshape(predictions_array, (-1, 1))

        return predictions_reshaped

In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random

In [52]:
import h5py
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.model_selection import cross_val_score
import time 
start_time = time.time()


file = h5py.File('DB2//DB2_S1_feature_200_0.h5','r')
featureData   = file['featureData'][:]
featureLabel  = file['featureLabel'][:]
file.close()

featureData = MinMaxScaler().fit_transform(featureData) # 缩放到[0, 1]
train_x, test_x, train_y, test_y = train_test_split(featureData, featureLabel, test_size=0.2)
print(train_y.shape)

(9531,)


新测试：

In [87]:
start = time.time()
RF = RandomForest(num_estimators = 10, 
                  max_depth = 12, min_samples_split = 1, numberSamplePerClass = 100)
RF.fit(train_x, train_y)


1


C:\Users\10062\AppData\Local\Temp\ipykernel_17508\3774368366.py:133: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  sorted_data = sorted(dataset, key=lambda x: x[feature])
C:\Users\10062\AppData\Local\Temp\ipykernel_17508\3774368366.py:137: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  left_data = np.array([row for row in sorted_data if row[feature] <= threshold])
C:\Users\10062\AppData\Local\Temp\ipykernel_17508\3774368366.py:138: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  right_data = np.array([row for row in sorted_data if row[feature] > threshold])


一棵树生成所用时间 5025.003315s!
2
一棵树生成所用时间 3794.260574s!
3
一棵树生成所用时间 3754.897275s!
4
一棵树生成所用时间 3649.704136s!
5
一棵树生成所用时间 3646.613616s!
6
一棵树生成所用时间 3893.775812s!
7
一棵树生成所用时间 3743.324507s!
8
一棵树生成所用时间 3403.643856s!
9
一棵树生成所用时间 3300.691520s!
10
一棵树生成所用时间 3841.123628s!


In [99]:
predictTest = RF.predict(test_x)

predictnew1 = predictTest.reshape(len(predictTest))
count1=0
for i in range(len(predictnew1)):
    if predictnew1[i] is None:
        #print(i,predictnew[i])
        count1=count1+1
        predictnew1[i]=0
count1=0
for i in range(len(predictnew1)):
    if predictnew1[i] == test_y[i]:
        count1 = count1+1

count1 = count1/(len(predictnew1))
print(count1)
print('10棵树/每个动作100个样本/最大深度12，花费时间: %.2f 秒。' % (time.time() - start))
print('RF test  accuracy: %.2f%%' % (100 * count1))

predictTrain = RF.predict(train_x)

predictnew2 = predictTrain.reshape(len(predictTrain))
count2=0
for i in range(len(predictnew2)):
    if predictnew2[i] is None:
        #print(i,predictnew[i])
        count2=count2+1
        predictnew2[i]=0
print(count)
count2=0
for i in range(len(predictnew2)):
    if predictnew2[i] == train_y[i]:
        count2 = count2+1

count2 = count2/(len(predictnew2))
print(count2)
print('10棵树/每个动作100个样本/最大深度12，花费时间: %.2f 秒。' % (time.time() - start))
print('RF train  accuracy: %.2f%%' % (100 * count2))

C:\Users\10062\AppData\Local\Temp\ipykernel_17508\3774368366.py:200: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if point.size > 0 and point[feature] <= threshold:                            #比较当前维度下未知点和阈值的大小，判断走左子树还是右子树


0.5644146034410407
10棵树/每个动作100个样本/最大深度12，花费时间: 41517.84 秒。
RF test  accuracy: 56.44%
0.6766341412233764
0.6766341412233764
10棵树/每个动作100个样本/最大深度12，花费时间: 41518.23 秒。
RF train  accuracy: 67.66%


对比研究，以下是sklearn中的随机森林模型

In [1]:
import numpy as np
import h5py
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import time

file = h5py.File('DB2//DB2_S1_feature_200_0.h5','r')
featureData   = file['featureData'][:]
featureLabel  = file['featureLabel'][:]
file.close()

featureData = MinMaxScaler().fit_transform(featureData) # 缩放到[0, 1]
trainData, testData, trainLabel, testLabel = train_test_split(featureData, featureLabel, test_size=0.2)



In [2]:
start_time = time.time()
RF2 = RandomForestClassifier(n_estimators=10, criterion='gini', max_depth=None, min_samples_split=2, 
                            min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='sqrt',
                            max_leaf_nodes=None, min_impurity_decrease=0.0,
                            bootstrap=True, oob_score=True, n_jobs=1, random_state=None, verbose=0,
                            warm_start=False, class_weight=None)


RF2.fit(trainData, trainLabel)
score2 = RF2.score(trainData, trainLabel)
predict2 = RF2.predict(testData)
accuracy2 = metrics.accuracy_score(testLabel, predict2)

print("RF train accuracy: %.2f%%" %(100*score2))
print('RF test  accuracy: %.2f%%' % (100 * accuracy2))
print ('training took %fs!' % (time.time() - start_time))

RF train accuracy: 99.63%
RF test  accuracy: 74.99%
training took 0.855669s!


D:\python\Lib\site-packages\sklearn\ensemble\_forest.py:615: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
